In [1]:
import pandas
import researchpy as rp
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp

In [2]:
location = r'E:\MYLEARN\2-ANALYTICS-DataScience\datasets\anova-2way-crop-yield.csv'

In [4]:
df = pandas.read_csv(location)
df.head()

,Fert,Water,Yield
0,A,High,27.4
1,A,High,33.6
2,A,High,29.8
3,A,High,35.2
4,A,High,33.0


In [5]:
rp.summary_cont(df['Yield'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,Yield,20.0,29.04,4.230516,0.945972,27.060058,31.019942


In [9]:
rp.summary_cont(df.groupby(['Fert']))['Yield']

D:\ANACONDA-8-APR\lib\site-packages\researchpy\summary.py:151: RuntimeWarning: invalid value encountered in sqrt
  l_ci = lambda x: numpy.mean(x) - (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))
D:\ANACONDA-8-APR\lib\site-packages\researchpy\summary.py:154: RuntimeWarning: invalid value encountered in sqrt
  u_ci = lambda x: numpy.mean(x) + (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))


,N,Mean,SD,SE,95% Conf.,Interval
Fert,,,,,,
A,10,30.90,3.283968,1.038482,28.864576,32.935424
B,10,27.18,4.394390,1.389628,24.456329,29.903671


In [7]:
rp.summary_cont(df.groupby(['Water']))['Yield']

D:\ANACONDA-8-APR\lib\site-packages\researchpy\summary.py:151: RuntimeWarning: invalid value encountered in sqrt
  l_ci = lambda x: numpy.mean(x) - (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))
D:\ANACONDA-8-APR\lib\site-packages\researchpy\summary.py:154: RuntimeWarning: invalid value encountered in sqrt
  u_ci = lambda x: numpy.mean(x) + (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))


,N,Mean,SD,SE,95% Conf.,Interval
Water,,,,,,
High,10,30.82,3.244756,1.026082,28.808879,32.831121
Low,10,27.26,4.495974,1.421752,24.473367,30.046633


In [10]:
rp.summary_cont(df.groupby(['Fert', 'Water']))['Yield']

D:\ANACONDA-8-APR\lib\site-packages\researchpy\summary.py:151: RuntimeWarning: invalid value encountered in sqrt
  l_ci = lambda x: numpy.mean(x) - (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))
D:\ANACONDA-8-APR\lib\site-packages\researchpy\summary.py:154: RuntimeWarning: invalid value encountered in sqrt
  u_ci = lambda x: numpy.mean(x) + (1.960 * (numpy.std(x)/numpy.sqrt(x.count() - 1)))


N   Mean        SD        SE  95% Conf.   Interval
Fert Water                                                    
A    High   5  31.80  3.146427  1.407125  29.042036  34.557964
     Low    5  30.00  3.512834  1.570987  26.920866  33.079134
B    High   5  29.84  3.374611  1.509172  26.882023  32.797977
     Low    5  24.52  3.791042  1.695406  21.197005  27.842995

In [11]:
# Fits the model with the interaction term
# This will also automatically include the main effects for each factor
model = ols('Yield ~ C(Fert) * C(Water)', df).fit()

In [21]:
# Seeing if the overall model is significant
# print(f"Overall model F({model.df_model: .0f}, {model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")

In [22]:
print('F-statistic : ', model.fvalue)
print('p-value     : ', model.f_pvalue)

F-statistic :  4.112444444444447
p-value     :  0.024302580837416664


In [23]:
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
C(Fert),69.192,1.0,5.766000,0.028847
C(Water),63.368,1.0,5.280667,0.035386
C(Fert):C(Water),15.488,1.0,1.290667,0.272656
Residual,192.000,16.0,NaN,NaN


- The interaction term is not significant. This indicates that there is no interaction effect between the type of __fertilizer__ and the __amount of water__ on the mean __crop yield__. Since this is not significant, the __interaction term__ is to be removed from the model and it needs to be re-ran so we can look at the main effects of each factor independently.

In [25]:
# Fits the model
model2 = ols('Yield ~ C(Fert)+ C(Water)', df).fit()

print(f"Overall model F({model2.df_model: .0f},{model2.df_resid: .0f}) = {model2.fvalue: .3f}, p = {model2.f_pvalue: .4f}")

Overall model F( 2, 17) =  5.430, p =  0.0150


In [26]:
# Creates the ANOVA table
res2 = sm.stats.anova_lm(model2, typ= 2)
res2

,sum_sq,df,F,PR(>F)
C(Fert),69.192,1.0,5.669070,0.029228
C(Water),63.368,1.0,5.191895,0.035887
Residual,207.488,17.0,NaN,NaN


Each factor has an independent significant effect on the mean crop yield.

#### check the __assumptions__ of the ANOVA, normality and homogeneity of variance.

In [28]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Yield   R-squared:                       0.390
Model:                            OLS   Adj. R-squared:                  0.318
Method:                 Least Squares   F-statistic:                     5.430
Date:                Mon, 04 May 2020   Prob (F-statistic):             0.0150
Time:                        12:14:48   Log-Likelihood:                -51.772
No. Observations:                  20   AIC:                             109.5
Df Residuals:                      17   BIC:                             112.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          32.6800      1.353     24.153      0.000      29.825      35.535
C(Fert)[T.B]       -3.7200      1.562     -2.381      0.029      -7.016      -0.424
C(Water)[T.Low]    -3.5600      1.562     -2.279      0.036      -6.856      -0.264
==============================================================================
Omnibus:                        1.169   Durbin-Watson:                   2.736
Prob(Omnibus):                  0.557   Jarque-Bera (JB):                0.820
Skew:                          -0.081   Prob(JB):                        0.664
Kurtosis:                       2.022   Cond. No.                         3.19
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

- The __Durban-Watson__ tests is to detect the presence of __autocorrelation__
- __Jarque-Bera__ tests the assumption of __normality__
- __Omnibus tests__ the assumption of __homogeneity of variance__ 
- the __Condition Number__ assess __multicollinearity__ 
    - Condition Number values over 20 are indicative of multicollinearity. 
    
The model passes the assumption check 

## Post-hoc Testing

Tukey’s HSD. 

We have to test for difference for each factor separately.

In [29]:
mc = statsmodels.stats.multicomp.MultiComparison(df['Yield'], df['Fert'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     A      B    -3.72 0.0459 -7.3647 -0.0753   True
----------------------------------------------------


There is a statistically significant different between the mean crop yield between fertilizer A and B; 

fertilizer A yields a significantly higher crop yield than fertilizer B.

In [30]:
mc = statsmodels.stats.multicomp.MultiComparison(df['Yield'], df['Water'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
  High    Low    -3.56 0.0574 -7.2436 0.1236  False
---------------------------------------------------


There is not a statistically significant difference in the mean crop yield between the amount of water used.